In [18]:
#import modules
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pylab as plt
import seaborn as sns
import credentials
import datetime
from datetime import date
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [19]:
#access database
URI = "dublinbikeappdb.cxaxe40vwlui.us-east-1.rds.amazonaws.com"
DB = "dbikes1"
name = credentials.name
pw = credentials.password

"""Initate connection"""
engine = create_engine("mysql+mysqlconnector://{}:{}@{}:3306/{}".format(name,pw,URI,DB),echo=False)

In [20]:
#get infor from db (testing with station number 4)
n = 4
availability = '''SELECT *
FROM availability
WHERE number = {}
'''.format(n)

weather = '''SELECT *
FROM weather
'''

select = """SELECT s.number
FROM stations s"""

In [21]:
#convert db tables to pandas dataframes
dfs = pd.read_sql_query(select, engine)
dfa = pd.read_sql_query(availability, engine)
dfw = pd.read_sql_query(weather, engine)

In [22]:
#merge weather and occupancy tables together
df = pd.merge_asof(dfa, dfw, left_on="last_update", right_on="last_update")

In [23]:
#transfor features
df["time"] = df["last_update"]
df["day"] = df["last_update"]

for i in range(df.shape[0]):
    df["time"].loc[i]= df.iloc[i]["last_update"].hour
    df["day"].loc[i] = df.iloc[i]["last_update"].dayofweek
    x= df.iloc[i]["last_update"].dayofweek

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [7]:
# implement weather data
df["good_weather"] = np.logical_or(df["description"] == "Clouds", df["description"] == "Clear")
df.good_weather.replace({True:1, False:0},inplace=True)

In [8]:
# create test and train sets
X = df[["time","day","humidity","temp","good_weather"]]
y = df.available_bikes

X_train, X_test, y_train , y_test = train_test_split(X,y, test_size=0.3, random_state=0)

In [9]:
#linear model
model = LinearRegression()
model.fit(X_train, y_train)

print(model.coef_)
print(model.intercept_)

[-0.06287618 -0.85223211 -0.00776007 -0.12462405 -1.33553959]
13.293161787491588


In [10]:
#run test set
model_predicitons = model.predict(X_test)
actual_vs_predicted = pd.concat([y_test, pd.DataFrame(model_predicitons,columns=["Predicted"],index=y_test.index)],axis=1)

In [11]:
#compute error metrics for test set
actual_vs_predicted["squared_error"] = (actual_vs_predicted["available_bikes"] - actual_vs_predicted["Predicted"])**2
rmse_test = ((actual_vs_predicted["squared_error"].sum())/actual_vs_predicted.shape[0])**0.5
mae_test = ((actual_vs_predicted["squared_error"])**0.5).sum()/actual_vs_predicted.shape[0]

print("RMSE",rmse_test)
print("MAE",mae_test)

RMSE 3.944733890098263
MAE 3.077000434178689


In [12]:
#colate this for single function
def get_model(n):
    """returns a model with error scores using the test set"""
    #get infor from db
    availability = '''SELECT *
    FROM availability
    WHERE number = {}
    '''.format(n)

    weather = '''SELECT *
    FROM weather
    '''

    select = """SELECT s.number
    FROM stations s"""
    
    #convert db tables to pandas dataframes
    dfs = pd.read_sql_query(select, engine)
    dfa = pd.read_sql_query(availability, engine)
    dfw = pd.read_sql_query(weather, engine)
    
    #merge weather and occupancy tables together
    df = pd.merge_asof(dfa, dfw, left_on="last_update", right_on="last_update")
    
    #transform existing features
    df["time"] = df["last_update"]
    df["day"] = df["last_update"]

    for i in range(df.shape[0]):
        df["time"].loc[i]= df.iloc[i]["last_update"].hour
        df["day"].loc[i] = df.iloc[i]["last_update"].dayofweek
        
    df["good_weather"] = np.logical_or(df["description"] == "Clouds", df["description"] == "Clear")
    df.good_weather.replace({True:1, False:0},inplace=True)
    
    # create test and train sets
    X = df[["time","day","humidity","temp","good_weather"]]
    y = df.available_bikes

    X_train, X_test, y_train , y_test = train_test_split(X,y, test_size=0.3, random_state=0)
    
    #linear model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    #run test set
    model_predicitons = model.predict(X_test)
    actual_vs_predicted = pd.concat([y_test, pd.DataFrame(model_predicitons,columns=["Predicted"],index=y_test.index)],axis=1)
    
    actual_vs_predicted["squared_error"] = (actual_vs_predicted["available_bikes"] - actual_vs_predicted["Predicted"])**2
    rmse_test = ((actual_vs_predicted["squared_error"].sum())/actual_vs_predicted.shape[0])**0.5
    mae_test = ((actual_vs_predicted["squared_error"])**0.5).sum()/actual_vs_predicted.shape[0]
    
    #get model using full dataset
    model = LinearRegression()
    model.fit(X, y)
    results = [model.coef_,model.intercept_,{"rmse":rmse_test, "mae":mae_test}]
    
    return results
                

In [13]:
stations = dfs.number.tolist()
stations[:3]

[2, 3, 4]

In [16]:
#trains all models, and returns error scores
#WARNING. THIS FUNCTION WILL TAKE A LONG TIME AS YOU ARE RUNNING ALL CODE FEATURED IN THIS NOTEBOOK FOR
#EVERY MODE IN THE DATASET. However long it took you to run all code thus far, this code will repeat those task
#for over 100 stations instead of just 1.
models = {}
for s in stations:
    models[s] = get_model(s)
    print("Station number :",s," Complete")

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

In [17]:
models 

{2: [array([ 0.04346646, -0.63703897,  0.07828144, -0.04983062, -0.46204519]),
  2.0103414341393337,
  {'rmse': 3.8097224340378624, 'mae': 3.1935679296398107}],
 3: [array([ 0.01732284,  0.04287028, -0.00870872, -0.15621976, -1.33683882]),
  7.812916258604407,
  {'rmse': 4.186200084591943, 'mae': 3.5088651828753576}],
 4: [array([-0.062267  , -0.87094248, -0.00678824, -0.11498882, -1.25518705]),
  13.147656033030778,
  {'rmse': 3.944733890098263, 'mae': 3.077000434178689}],
 5: [array([-0.02204101, -0.81156563, -0.21434746, -0.70982689,  3.09298629]),
  36.58433783138004,
  {'rmse': 8.664677947021588, 'mae': 6.755259881603461}],
 6: [array([ 0.04951701,  0.12205287, -0.03776422, -0.03881449, -0.67204275]),
  5.401499667505921,
  {'rmse': 2.7586805647842962, 'mae': 2.192927426927}],
 7: [array([ 0.04262032,  0.37202214, -0.08560877, -0.45623201,  1.45269336]),
  14.094950462735518,
  {'rmse': 4.740800840947765, 'mae': 3.730863285053955}],
 8: [array([-0.06033067, -0.44067477, -0.0311315